<a href="https://colab.research.google.com/github/NBene10/cross-domain_SentimentAnalysis/blob/main/Cross_Domain_SentimentAnlysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
!pip install vadersentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re
from nltk.tokenize import word_tokenize
import nltk


Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.1 MB/s eta 0:00:00


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/AI/datasets/drugsComTrain_raw.tsv", delimiter="\t",encoding='utf-8')

In [ ]:
df.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37


In [ ]:

print("Dataframe total NaN values: ")
df.isnull().T.any().T.sum()


Dataframe total NaN values: 


899

In [ ]:
#dataset removing NaN value
df.fillna('', inplace=True)
print("Dataframe total NaN values: ")
df.isnull().T.any().T.sum()

Dataframe total NaN values: 


0

In [ ]:
#text pre-processing dataset column 'review'

nltk.download('stopwords')
nltk.download('punkt')

def preprocess_text(text):

    #Remove non-alphanumeric characters and convert to lowercase
    text = re.sub(r'[^a-zA-Z0-9]', ' ', text.lower())

    #Tokenizing and removing stop-words
    stop_words = set(stopwords.words('english'))
    words = word_tokenize(text)
    filtered_words = [word for word in words if word not in stop_words]

    return ' '.join(filtered_words)

df['cleaned_review'] = df['review'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,cleaned_review
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,side effect take combination bystolic 5 mg fis...
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,son halfway fourth week intuniv became concern...
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,used take another oral contraceptive 21 pill c...
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,first time using form birth control 039 glad w...
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,suboxone completely turned life around feel he...


In [ ]:
analyser = SentimentIntensityAnalyzer()

#analyser for cleaned review
df['cleaned_review-dict-VD'] = df['cleaned_review'].apply(analyser.polarity_scores)
df['cleaned_review-numberlist-VD'] = df['cleaned_review-dict-VD'].apply(lambda x: x.values())

#analyser for standard review
df['standard_review-dict-VD'] = df['review'].apply(analyser.polarity_scores)
df['standard_review-numberlist-VD'] = df['standard_review-dict-VD'].apply(lambda x: x.values())

dfcleanVD = df['cleaned_review-numberlist-VD']
dfstandardVD = df['standard_review-numberlist-VD']

dfcleanVD = pd.DataFrame(dfcleanVD.tolist(), columns=['key_neg', 'key_neu', 'key_pos', 'key_compound'])
dfstandardVD = pd.DataFrame(dfstandardVD.tolist(), columns=['key_neg', 'key_neu', 'key_pos', 'key_compound'])


In [ ]:
df.head()

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,cleaned_review,cleaned_review-dict-VD,cleaned_review-numberlist-VD,standard_review-dict-VD,standard_review-numberlist-VD
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,side effect take combination bystolic 5 mg fis...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","(0.0, 1.0, 0.0, 0.0)","{'neg': 0.121, 'neu': 0.879, 'pos': 0.0, 'comp...","(0.121, 0.879, 0.0, -0.296)"
1,95260,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,"April 27, 2010",192,son halfway fourth week intuniv became concern...,"{'neg': 0.035, 'neu': 0.759, 'pos': 0.206, 'co...","(0.035, 0.759, 0.206, 0.9068)","{'neg': 0.018, 'neu': 0.874, 'pos': 0.108, 'co...","(0.018, 0.874, 0.108, 0.9174)"
2,92703,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,"December 14, 2009",17,used take another oral contraceptive 21 pill c...,"{'neg': 0.062, 'neu': 0.811, 'pos': 0.126, 'co...","(0.062, 0.811, 0.126, 0.7506)","{'neg': 0.059, 'neu': 0.861, 'pos': 0.08, 'com...","(0.059, 0.861, 0.08, 0.616)"
3,138000,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,"November 3, 2015",10,first time using form birth control 039 glad w...,"{'neg': 0.039, 'neu': 0.804, 'pos': 0.157, 'co...","(0.039, 0.804, 0.157, 0.7184)","{'neg': 0.026, 'neu': 0.885, 'pos': 0.089, 'co...","(0.026, 0.885, 0.089, 0.7184)"
4,35696,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,"November 27, 2016",37,suboxone completely turned life around feel he...,"{'neg': 0.105, 'neu': 0.612, 'pos': 0.283, 'co...","(0.105, 0.612, 0.283, 0.9344)","{'neg': 0.061, 'neu': 0.771, 'pos': 0.168, 'co...","(0.061, 0.771, 0.168, 0.9403)"


In [ ]:
dfcleanVD.head()

,key_neg,key_neu,key_pos,key_compound
0,0.000,1.000,0.000,0.0000
1,0.035,0.759,0.206,0.9068
2,0.062,0.811,0.126,0.7506
3,0.039,0.804,0.157,0.7184
4,0.105,0.612,0.283,0.9344


In [ ]:
dfstandardVD.head()

,key_neg,key_neu,key_pos,key_compound
0,0.121,0.879,0.000,-0.2960
1,0.018,0.874,0.108,0.9174
2,0.059,0.861,0.080,0.6160
3,0.026,0.885,0.089,0.7184
4,0.061,0.771,0.168,0.9403


In [ ]:
dfclean_final = pd.merge(df, dfcleanVD, left_index=True, right_index=True)
dfclean_final.drop(columns=['standard_review-dict-VD'], inplace=True)
dfclean_final.drop(columns=['standard_review-numberlist-VD'], inplace=True)

In [ ]:
dfclean_final.head(1)

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,cleaned_review,cleaned_review-dict-VD,cleaned_review-numberlist-VD,key_neg,key_neu,key_pos,key_compound
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,side effect take combination bystolic 5 mg fis...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","(0.0, 1.0, 0.0, 0.0)",0.0,1.0,0.0,0.0


In [ ]:
dfstandard_final = pd.merge(df, dfstandardVD, left_index=True, right_index=True)
dfstandard_final.drop(columns=['cleaned_review'], inplace=True)
dfstandard_final.drop(columns=['cleaned_review-dict-VD'], inplace=True)
dfstandard_final.drop(columns=['cleaned_review-numberlist-VD'], inplace=True)

In [ ]:
dfstandard_final.head(1)

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,standard_review-dict-VD,standard_review-numberlist-VD,key_neg,key_neu,key_pos,key_compound
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,"{'neg': 0.121, 'neu': 0.879, 'pos': 0.0, 'comp...","(0.121, 0.879, 0.0, -0.296)",0.121,0.879,0.0,-0.296


In [ ]:
# Definisci una funzione per applicare i criteri alla colonna "ratings_cond"
def categorize_ratings(rating):
    if 1 <= rating <= 4:
        return "1 <= rating <= 4"
    elif 5 <= rating <= 6:
        return "4 < rating < 7"
    elif rating >= 7:
        return "rating >= 7"
    else:
        return None

dfclean_final['ratings_cond'] = df['rating'].apply(lambda x: categorize_ratings(x))
dfstandard_final['ratings_cond'] = df['rating'].apply(lambda x: categorize_ratings(x))

# Definisci una funzione per mappare i valori di "ratings_cond" a "rating_model"
def map_rating_model(ratings_cond):
    if ratings_cond == "1 <= rating <= 4":
        return 0
    elif ratings_cond == "4 < rating < 7":
        return 1
    elif ratings_cond == "rating >= 7":
        return 2
    else:
        return None  # Trattamento per valori non previsti


# Applica la funzione alla colonna "ratings_cond" e crea la nuova colonna "rating_model"
dfclean_final['rating_model'] = dfclean_final['ratings_cond'].apply(lambda x: map_rating_model(x))
dfstandard_final['rating_model'] = dfstandard_final['ratings_cond'].apply(lambda x: map_rating_model(x))


In [ ]:
dfclean_final.head(1)

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,cleaned_review,cleaned_review-dict-VD,cleaned_review-numberlist-VD,key_neg,key_neu,key_pos,key_compound,ratings_cond,rating_model
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,side effect take combination bystolic 5 mg fis...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...","(0.0, 1.0, 0.0, 0.0)",0.0,1.0,0.0,0.0,rating >= 7,2


In [ ]:
dfstandard_final.head(1)

,Unnamed: 0,drugName,condition,review,rating,date,usefulCount,standard_review-dict-VD,standard_review-numberlist-VD,key_neg,key_neu,key_pos,key_compound,ratings_cond,rating_model
0,206461,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,"May 20, 2012",27,"{'neg': 0.121, 'neu': 0.879, 'pos': 0.0, 'comp...","(0.121, 0.879, 0.0, -0.296)",0.121,0.879,0.0,-0.296,rating >= 7,2


In [ ]:
# Boolean Mask with conditions, ISIN, wenn mehrere conditions abgefragt werden

conditions=['Birth Control', 'Depression','Pain', 'Anxiety', 'Diabetes, Type 2']
def model_data_clean(condition):
    dfclean_CrossDomain = dfclean_final[dfclean_final['condition'] == condition].copy()
    print(len(dfclean_CrossDomain))
    X_trainCl, X_testCl, y_trainCl, y_testCl = train_test_split(dfclean_CrossDomain['cleaned_review'], dfclean_CrossDomain['rating_model'], test_size=0.33, random_state=42)
    return X_trainCl, X_testCl, y_trainCl, y_testCl

def model_data_standard(condition):
    dfstandard_CrossDomain = dfstandard_final[dfstandard_final['condition'] == condition].copy()
    print(len(dfstandard_CrossDomain))
    X_trainSt, X_testSt, y_trainSt, y_testSt = train_test_split(dfstandard_CrossDomain['review'], dfstandard_CrossDomain['rating_model'], test_size=0.33, random_state=42)
    return X_trainSt, X_testSt, y_trainSt, y_testSt


In [ ]:
X_BC_trainCl, X_BC_testCl, y_BC_trainCl, y_BC_testCl = model_data_clean('Birth Control')
X_D_trainCl, X_D_testCl, y_D_trainCl, y_D_testCl = model_data_clean('Depression')
X_P_trainCl, X_P_testCl, y_P_trainCl, y_P_testCl = model_data_clean('Pain')
X_A_trainCl, X_A_testCl, y_A_trainCl, y_A_testCl = model_data_clean('Anxiety')
X_Dia_trainCl, X_Dia_testCl, y_Dia_trainCl, y_Dia_testCl = model_data_clean('Diabetes, Type 2')

28788
9069
6145
5904
2554


In [ ]:
X_BC_trainSt, X_BC_testSt, y_BC_trainSt, y_BC_testSt = model_data_standard('Birth Control')
X_D_trainSt, X_D_testSt, y_D_trainSt, y_D_testSt = model_data_standard('Depression')
X_P_trainSt, X_P_testSt, y_P_trainSt, y_P_testSt = model_data_standard('Pain')
X_A_trainSt, X_A_testSt, y_A_trainSt, y_A_testSt = model_data_standard('Anxiety')
X_Dia_trainSt, X_Dia_testSt, y_Dia_trainSt, y_Dia_testSt = model_data_standard('Diabetes, Type 2')

28788
9069
6145
5904
2554


In [ ]:
trainCl_str_Inputs = [['X_BC_trainCl', 'y_BC_trainCl'],['X_D_trainCl', 'y_D_trainCl'], ['X_P_trainCl','y_P_trainCl'], ['X_A_trainCl','y_A_trainCl'],['X_Dia_trainCl','y_Dia_trainCl']]
testCl_str_Inputs = [['X_BC_testCl','y_BC_testCl'], ['X_D_testCl','y_D_testCl'],['X_P_testCl','y_P_tesCl'],['X_A_testCl','y_A_testCl'],['X_Dia_testCl','y_Dia_testCl']]
trainSt_str_Inputs = [['X_BC_trainSt', 'y_BC_trainSt'],['X_D_trainSt', 'y_D_trainSt'], ['X_P_trainSt','y_P_trainSt'], ['X_A_trainSt','y_A_trainSt'],['X_Dia_trainSt','y_Dia_trainSt']]
testSt_str_Inputs = [['X_BC_testSt','y_BC_testSt'], ['X_D_testSt','y_D_testSt'],['X_P_testSt','y_P_tesSt'],['X_A_testSt','y_A_testSt'],['X_Dia_testSt','y_Dia_testSt']]

string_combinationCl= [t+j for j in testCl_str_Inputs for t in trainCl_str_Inputs]
string_combinationSt= [t+j for j in testSt_str_Inputs for t in trainSt_str_Inputs]

trainCl_var_Inputs= [(X_BC_trainCl, y_BC_trainCl),(X_D_trainCl, y_D_trainCl), (X_P_trainCl,y_P_trainCl), (X_A_trainCl,y_A_trainCl),(X_Dia_trainCl,y_Dia_trainCl)]
testCl_var_Inputs = [(X_BC_testCl,y_BC_testCl), (X_D_testCl,y_D_testCl),(X_P_testCl,y_P_testCl),(X_A_testCl,y_A_testCl),(X_Dia_testCl,y_Dia_testCl)]
varCl_combination= [t+j for j in testCl_var_Inputs for t in trainCl_var_Inputs]
trainSt_var_Inputs= [(X_BC_trainSt, y_BC_trainSt),(X_D_trainSt, y_D_trainSt), (X_P_trainSt,y_P_trainSt), (X_A_trainSt,y_A_trainSt),(X_Dia_trainSt,y_Dia_trainSt)]
testSt_var_Inputs = [(X_BC_testSt,y_BC_testSt), (X_D_testSt,y_D_testSt),(X_P_testSt,y_P_testSt),(X_A_testSt,y_A_testSt),(X_Dia_testSt,y_Dia_testSt)]
varSt_combination= [t+j for j in testSt_var_Inputs for t in trainSt_var_Inputs]

In [ ]:

# Definizione dei Classificatori
Classifiers = [
    Perceptron(),
]


v = TfidfVectorizer(analyzer="word", ngram_range=(1, 2))
# Inizializzazione del Dizionario e Ciclo Principale
dicti_Cl = {"train": [], "test": [], "accuracy": []}
dicti_St = {"train": [], "test": [], "accuracy": []}

print("[PERCEPTRON ON CLEAN DF]")
for i, (X_trainCl, y_trainCl, X_testCl, y_testCl) in zip(string_combinationCl, varCl_combination):
    print('-' * 100)
    print(i)
    print('-' * 100)

    # Estrazione delle Caratteristiche e Addestramento del Modello
    train_featuresCl = v.fit_transform(X_trainCl)
    test_featuresCl = v.transform(X_testCl)

    if train_featuresCl.shape[1] != test_featuresCl.shape[1]:
        print(f"Error: Dimension mismatch between training and test features for {i}")
        continue

    for classifier in Classifiers:
        try:
            fit = classifier.fit(train_featuresCl, y_trainCl)
            pred = fit.predict(test_featuresCl)
            accuracy = accuracy_score(pred, y_testCl)

            # Registrazione dei Risultati nel Dizionario
            dicti_Cl["train"].append(i[0][2:4])
            dicti_Cl["test"].append(i[3][2:4])
            dicti_Cl["accuracy"].append(float(accuracy))
            print("Accuracy of " + classifier.__class__.__name__ + " is " + str(accuracy))
        except Exception as e:
            print(f"Error: {str(e)}")

print("\n[PERCEPTRON ON STANDARD DF]")
for i, (X_trainSt, y_trainSt, X_testSt, y_testSt) in zip(string_combinationSt, varSt_combination):
    print('-' * 100)
    print(i)
    print('-' * 100)


    train_featuresSt = v.fit_transform(X_trainSt)
    test_featuresSt = v.transform(X_testSt)

    for classifier in Classifiers:
        try:
            fit = classifier.fit(train_featuresSt, y_trainSt)
            pred = fit.predict(test_featuresSt)
            accuracy = accuracy_score(pred, y_testSt)

            # Registrazione dei Risultati nel Dizionario
            dicti_St["train"].append(i[0][2:4])
            dicti_St["test"].append(i[3][2:4])
            dicti_St["accuracy"].append(float(accuracy))
            print("Accuracy of " + classifier.__class__.__name__ + " is " + str(accuracy))
        except Exception as e:
            print(f"Error: {str(e)}")



[PERCEPTRON ON CLEAN DF]
----------------------------------------------------------------------------------------------------
['X_BC_trainCl', 'y_BC_trainCl', 'X_BC_testCl', 'y_BC_testCl']
----------------------------------------------------------------------------------------------------
Accuracy of Perceptron is 0.8790653615408904
----------------------------------------------------------------------------------------------------
['X_D_trainCl', 'y_D_trainCl', 'X_BC_testCl', 'y_BC_testCl']
----------------------------------------------------------------------------------------------------
Accuracy of Perceptron is 0.6147773918534891
----------------------------------------------------------------------------------------------------
['X_P_trainCl', 'y_P_trainCl', 'X_BC_testCl', 'y_BC_testCl']
----------------------------------------------------------------------------------------------------
Accuracy of Perceptron is 0.5801494579517945
-------------------------------------------------

In [ ]:
print(dicti_Cl)

{'train': ['BC', 'D_', 'P_', 'A_', 'Di', 'BC', 'D_', 'P_', 'A_', 'Di', 'BC', 'D_', 'P_', 'A_', 'Di', 'BC', 'D_', 'P_', 'A_', 'Di', 'BC', 'D_', 'P_', 'A_', 'Di'], 'test': ['BC', 'BC', 'BC', 'BC', 'BC', 'D_', 'D_', 'D_', 'D_', 'D_', 'P_', 'P_', 'P_', 'P_', 'P_', 'A_', 'A_', 'A_', 'A_', 'A_', 'Di', 'Di', 'Di', 'Di', 'Di'], 'accuracy': [0.8790653615408904, 0.6147773918534891, 0.5801494579517945, 0.5893063888011788, 0.580044205873066, 0.5242231874373539, 0.8559973270965586, 0.6906114266622119, 0.7236886067490812, 0.5215502839959907, 0.5749506903353058, 0.7199211045364892, 0.8742603550295858, 0.7840236686390533, 0.47830374753451677, 0.5089789635710621, 0.8142637249871729, 0.7583376090302719, 0.8722421754746024, 0.5105182144689584, 0.5931198102016607, 0.6868327402135231, 0.6358244365361803, 0.6654804270462633, 0.8552787663107948]}


In [ ]:
print(dicti_St)

{'train': ['BC', 'D_', 'P_', 'A_', 'Di', 'BC', 'D_', 'P_', 'A_', 'Di', 'BC', 'D_', 'P_', 'A_', 'Di', 'BC', 'D_', 'P_', 'A_', 'Di', 'BC', 'D_', 'P_', 'A_', 'Di'], 'test': ['BC', 'BC', 'BC', 'BC', 'BC', 'D_', 'D_', 'D_', 'D_', 'D_', 'P_', 'P_', 'P_', 'P_', 'P_', 'A_', 'A_', 'A_', 'A_', 'A_', 'Di', 'Di', 'Di', 'Di', 'Di'], 'accuracy': [0.8824334280602042, 0.6423534364803705, 0.594463740658878, 0.5955162614461635, 0.6131986106725608, 0.6004009355162044, 0.861343133979285, 0.7300367524223187, 0.7557634480454394, 0.6171065820247243, 0.6503944773175543, 0.7712031558185405, 0.8747534516765286, 0.7924063116370809, 0.5478303747534516, 0.5946639302206259, 0.8245253976398152, 0.7855310415597743, 0.8773730118009235, 0.6300667008722421, 0.6571767497034401, 0.708185053380783, 0.6820877817319099, 0.7034400948991696, 0.8469750889679716]}


In [ ]:
cross_domain = pd.DataFrame.from_dict(dicti_Cl) #the "cross_domain" DataFrame is created using the data contained in the "dicti_Cl" dictionary
cross_domain['test'].unique()

abbreviation = ['BC', 'D_', 'P_', 'A_', 'Di']

cross_domain.replace(to_replace=abbreviation, value=conditions, inplace=True) #replaces the abbreviations present in the "cross_domain" DataFrame with the full names of
                                                                              #the domains, using the "abbreviation" and "conditions" lists.

In [ ]:
cross_domain_perceptron = cross_domain.pivot_table(values='accuracy', index= 'train', columns='test')

cross_domain_perceptron

test,Anxiety,Birth Control,Depression,"Diabetes, Type 2",Pain
train,,,,,
Anxiety,0.872242,0.589306,0.723689,0.665480,0.784024
Birth Control,0.508979,0.879065,0.524223,0.593120,0.574951
Depression,0.814264,0.614777,0.855997,0.686833,0.719921
"Diabetes, Type 2",0.510518,0.580044,0.521550,0.855279,0.478304
Pain,0.758338,0.580149,0.690611,0.635824,0.874260


In [ ]:
cross_domain_perceptron_s = cross_domain_perceptron.reindex(index=['Depression', 'Anxiety','Pain', 'Birth Control','Diabetes, Type 2' ],columns= ['Depression', 'Anxiety','Pain', 'Birth Control','Diabetes, Type 2' ])
cross_domain_perceptron_s['|Train(Avg. accuracy)|'] = cross_domain_perceptron_s[cross_domain_perceptron_s.columns[::-1]].mean(axis=1)
cross_domain_perceptron_s.loc['|Test(Avg. accuracy)|'] = cross_domain_perceptron_s[cross_domain_perceptron_s.columns[::-1]].mean(axis=0)
cross_domain_perceptron_s

test,Depression,Anxiety,Pain,Birth Control,"Diabetes, Type 2",|Train(Avg. accuracy)|
train,,,,,,
Depression,0.855997,0.814264,0.719921,0.614777,0.686833,0.738358
Anxiety,0.723689,0.872242,0.784024,0.589306,0.665480,0.726948
Pain,0.690611,0.758338,0.874260,0.580149,0.635824,0.707837
Birth Control,0.524223,0.508979,0.574951,0.879065,0.593120,0.616068
"Diabetes, Type 2",0.521550,0.510518,0.478304,0.580044,0.855279,0.589139
|Test(Avg. accuracy)|,0.663214,0.692868,0.686292,0.648669,0.687307,0.675670


In [ ]:
cross_domain= pd.DataFrame.from_dict(dicti_St)
cross_domain['test'].unique()

abbreviation= ['BC', 'D_', 'P_', 'A_', 'Di']

cross_domain.replace(to_replace=abbreviation, value=conditions,inplace=True)

In [ ]:
cross_domain_perceptron = cross_domain.pivot_table(values='accuracy', index= 'train', columns='test')

cross_domain_perceptron

test,Anxiety,Birth Control,Depression,"Diabetes, Type 2",Pain
train,,,,,
Anxiety,0.877373,0.595516,0.755763,0.703440,0.792406
Birth Control,0.594664,0.882433,0.600401,0.657177,0.650394
Depression,0.824525,0.642353,0.861343,0.708185,0.771203
"Diabetes, Type 2",0.630067,0.613199,0.617107,0.846975,0.547830
Pain,0.785531,0.594464,0.730037,0.682088,0.874753
